# Data Preprocessing
Step 1: Conver datetime feild to_datetime for all datafiles
Step 2: Merge Machines data to telemetry data

In [3]:
import tensorflow, keras
import pandas
import numpy

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM

Using TensorFlow backend.


In [ ]:
#errors = pandas.read_csv('/Users/abhi/Desktop/Sem 4/BNAD/SQL-Server-R-Services-Samples/PredictiveMaintanenceModelingGuide/Data/errors.csv')
#failures = pandas.read_csv('/Users/abhi/Desktop/Sem 4/BNAD/SQL-Server-R-Services-Samples/PredictiveMaintanenceModelingGuide/Data/failures.csv')
#machines = pandas.read_csv('/Users/abhi/Desktop/Sem 4/BNAD/SQL-Server-R-Services-Samples/PredictiveMaintanenceModelingGuide/Data/machines.csv')
maintain = pandas.read_csv('/Users/abhi/Desktop/Sem 4/BNAD/SQL-Server-R-Services-Samples/PredictiveMaintanenceModelingGuide/Data/maint.csv')
telemetry = pandas.read_csv('/Users/abhi/Desktop/Sem 4/BNAD/SQL-Server-R-Services-Samples/PredictiveMaintanenceModelingGuide/Data/telemetry.csv')

In [ ]:
errors['datetime']=pandas.to_datetime(errors['datetime'])
telemetry['datetime']=pandas.to_datetime(telemetry['datetime'])
maintain['datetime']=pandas.to_datetime(maintain['datetime'])
failures['datetime']=pandas.to_datetime(failures['datetime'])

# Maintain pivot

In [ ]:
comp4 = maintain.iloc[numpy.where(maintain['comp']=='comp4')]
comp3 = maintain.iloc[numpy.where(maintain['comp']=='comp3')]
comp2 = maintain.iloc[numpy.where(maintain['comp']=='comp2')]
comp1 = maintain.iloc[numpy.where(maintain['comp']=='comp1')]

In [ ]:
maintain_pivot_t = (pandas.merge(comp4, comp3, on = ['datetime', 'machineID'], how = 'outer'))
maintain_pivot_t = maintain_pivot_t.rename(columns={maintain_pivot_t.columns[2]:'comp4', maintain_pivot_t.columns[3]:'comp3'})

In [ ]:
maintain_pivot_t2= pandas.merge(maintain_pivot_t, comp2, on = ['datetime', 'machineID'], how = 'outer')

In [ ]:
maintain_pivot= pandas.merge(maintain_pivot_t2, comp1, on = ['datetime', 'machineID'], how = 'outer')


In [ ]:
maintain_pivot.rename(columns={maintain_pivot.columns[2]:'comp4', maintain_pivot.columns[3]:'comp3'})

In [ ]:
maintain_pivot = maintain_pivot.fillna(0)
maintain_pivot.loc[maintain_pivot['comp1'] == 'comp1', 'comp1'] = 1
maintain_pivot.loc[maintain_pivot['comp2'] == 'comp2', 'comp2'] = 1
maintain_pivot.loc[maintain_pivot['comp3'] == 'comp3', 'comp3'] = 1
maintain_pivot.loc[maintain_pivot['comp4'] == 'comp4', 'comp4'] = 1

In [ ]:
maintain_pivot = maintain_pivot.rename(columns={maintain_pivot.columns[2]:'Mantain_comp4',maintain_pivot.columns[3]:'Mantain_comp3',maintain_pivot.columns[4]:'Mantain_comp2', maintain_pivot.columns[5]:'Mantain_comp1'})

In [ ]:
maintain_pivot

# Error Pivot

In [ ]:
error4 = errors.iloc[numpy.where(errors['errorID']=='error4')]
error3 = errors.iloc[numpy.where(errors['errorID']=='error3')]
error2 = errors.iloc[numpy.where(errors['errorID']=='error2')]
error1 = errors.iloc[numpy.where(errors['errorID']=='error1')]

In [ ]:
errors_pivot_t = (pandas.merge(error4, error3, on = ['datetime', 'machineID'], how = 'outer'))
errors_pivot_t = errors_pivot_t.rename(columns={errors_pivot_t.columns[2]:'error4', errors_pivot_t.columns[3]:'error3'})

In [ ]:
errors_pivot_t2= pandas.merge(errors_pivot_t, error2, on = ['datetime', 'machineID'], how = 'outer')
errors_pivot= pandas.merge(errors_pivot_t2, error1, on = ['datetime', 'machineID'], how = 'outer')

In [ ]:
errors_pivot = errors_pivot.rename(columns={errors_pivot.columns[4]:'error2', errors_pivot.columns[5]:'error1'})

In [ ]:
errors_pivot = errors_pivot.fillna(0)
errors_pivot.loc[errors_pivot['error1'] == 'error1', 'error1'] = 1
errors_pivot.loc[errors_pivot['error2'] == 'error2', 'error2'] = 1
errors_pivot.loc[errors_pivot['error3'] == 'error3', 'error3'] = 1
errors_pivot.loc[errors_pivot['error4'] == 'error4', 'error4'] = 1

In [ ]:
print(errors_pivot)

# failures Pivot

In [ ]:
comp4 = failures.iloc[numpy.where(failures['failure']=='comp4')]
comp3 = failures.iloc[numpy.where(failures['failure']=='comp3')]
comp2 = failures.iloc[numpy.where(failures['failure']=='comp2')]
comp1 = failures.iloc[numpy.where(failures['failure']=='comp1')]

In [ ]:
failures_pivot_t = (pandas.merge(comp4, comp3, on = ['datetime', 'machineID'], how = 'outer'))
failures_pivot_t = failures_pivot_t.rename(columns={failures_pivot_t.columns[2]:'comp4', failures_pivot_t.columns[3]:'comp3'})

In [ ]:
failures_pivot_t2= pandas.merge(failures_pivot_t, comp2, on = ['datetime', 'machineID'], how = 'outer')
failures_pivot= pandas.merge(failures_pivot_t2, comp1, on = ['datetime', 'machineID'], how = 'outer')

In [ ]:
failures_pivot = failures_pivot.rename(columns={failures_pivot.columns[4]:'comp2', failures_pivot.columns[5]:'comp1'})

In [ ]:
failures_pivot = failures_pivot.fillna(0)
failures_pivot.loc[failures_pivot['comp1'] == 'comp1', 'comp1'] = 1
failures_pivot.loc[failures_pivot['comp2'] == 'comp2', 'comp2'] = 1
failures_pivot.loc[failures_pivot['comp3'] == 'comp3', 'comp3'] = 1
failures_pivot.loc[failures_pivot['comp4'] == 'comp4', 'comp4'] = 1

In [ ]:
failures_pivot = failures_pivot.rename(columns={failures_pivot.columns[2]:'fail_comp4',failures_pivot.columns[3]:'fail_comp3',failures_pivot.columns[4]:'fail_comp2', failures_pivot.columns[5]:'fail_comp1'})

In [ ]:
failures_pivot

# Data combining process 

In [ ]:
fulldata = pandas.merge(telemetry, machines, on = ['machineID'], how = 'outer')


In [ ]:
fulldata

In [ ]:
fulldata1 = pandas.merge(fulldata, errors_pivot, on = ['datetime','machineID'], how = 'outer')
fulldata2 = pandas.merge(fulldata1, maintain_pivot, on = ['datetime','machineID'], how = 'outer')
fulldata3 = pandas.merge(fulldata2, failures_pivot, on = ['datetime','machineID'], how = 'outer')

In [ ]:
fulldata3 = fulldata3.fillna(0)

In [ ]:
print (telemetry.count())
print (errors.count())
print (maintain.count())
print (failures.count())
print(fulldata3.count())

In [ ]:
fulldata3.to_csv('final_sql_data.csv', sep = ',',encoding='utf-8', index=False)

# Read data and continue building Remaining Useful Life feature for failure and maintenance prediction

In [4]:
final_data = pandas.read_csv('final_sql_data.csv')

Process -
1. Add remaining useful life
   if failure exists and last maintenance exist or assume start of time ie first entry of data for perticular machine in telemetry
   Failure of component - Last Maintenance of component
2. generate label columns for training data
   we will only make use of "label1" for binary classification, 
   while trying to answer the question: is a specific engine going to fail within w1 cycles?
   w1 = 30
    w0 = 15
    train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
    train_df['label2'] = train_df['label1']
    train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
    

In [7]:
final_data = final_data.sort_values('datetime')

In [9]:
final_data

,datetime,machineID,volt,rotate,pressure,vibration,model,age,error4,error3,error2,error1,Mantain_comp4,Mantain_comp3,Mantain_comp2,Mantain_comp1,fail_comp4,fail_comp3,fail_comp2,fail_comp1
8761343,2014-06-01 06:00:00,344,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8761982,2014-06-01 06:00:00,983,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8762655,2014-06-01 06:00:00,690,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8761370,2014-06-01 06:00:00,371,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8764089,2014-06-01 06:00:00,336,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8763447,2014-06-01 06:00:00,581,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8761373,2014-06-01 06:00:00,374,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8763449,2014-06-01 06:00:00,583,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8761387,2014-06-01 06:00:00,388,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8763016,2014-06-01 06:00:00,79,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [34]:
final_data['DateTillMaintain_comp1'] =final_data[(final_data['Mantain_comp1']==1)].datetime
final_data['DateTillMaintain_comp2'] =final_data[(final_data['Mantain_comp2']==1)].datetime
final_data['DateTillMaintain_comp3'] =final_data[(final_data['Mantain_comp3']==1)].datetime
final_data['DateTillMaintain_comp4'] =final_data[(final_data['Mantain_comp4']==1)].datetime

final_data['DateTillFailure_comp1'] =final_data[(final_data['fail_comp1']==1)].datetime
final_data['DateTillFailure_comp2'] =final_data[(final_data['fail_comp2']==1)].datetime
final_data['DateTillFailure_comp3'] =final_data[(final_data['fail_comp3']==1)].datetime
final_data['DateTillFailure_comp4'] =final_data[(final_data['fail_comp4']==1)].datetime


## Not working as desired!!!! time series giving negative response both next repair and next failure

In [35]:
final_data = final_data.fillna(method='bfill')

In [36]:

final_data ['RULtillFail_comp1'] = (pandas.to_datetime(final_data['DateTillFailure_comp1'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')
final_data ['RULtillFail_comp2'] = (pandas.to_datetime(final_data['DateTillFailure_comp2'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')
final_data ['RULtillFail_comp3'] = (pandas.to_datetime(final_data['DateTillFailure_comp3'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')
final_data ['RULtillFail_comp4'] = (pandas.to_datetime(final_data['DateTillFailure_comp4'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')

final_data ['RULtillMaintain_comp1'] = (pandas.to_datetime(final_data['DateTillMaintain_comp1'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')
final_data ['RULtillMaintain_comp2'] = (pandas.to_datetime(final_data['DateTillMaintain_comp2'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')
final_data ['RULtillMaintain_comp3'] = (pandas.to_datetime(final_data['DateTillMaintain_comp3'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')
final_data ['RULtillMaintain_comp4'] = (pandas.to_datetime(final_data['DateTillMaintain_comp4'])- pandas.to_datetime(final_data['datetime'])).astype('timedelta64[h]')



In [37]:
final_data.columns

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'model', 'age', 'error4', 'error3', 'error2', 'error1', 'Mantain_comp4',
       'Mantain_comp3', 'Mantain_comp2', 'Mantain_comp1', 'fail_comp4',
       'fail_comp3', 'fail_comp2', 'fail_comp1', 'DateTillMaintain_comp1',
       'DateTillMaintain_comp2', 'DateTillMaintain_comp3',
       'DateTillMaintain_comp4', 'DateTillFailure_comp1',
       'DateTillFailure_comp2', 'DateTillFailure_comp3',
       'DateTillFailure_comp4', 'RULtillFail_comp1', 'RULtillFail_comp2',
       'RULtillFail_comp3', 'RULtillFail_comp4', 'RULtillMaintain_comp1',
       'RULtillMaintain_comp2', 'RULtillMaintain_comp3',
       'RULtillMaintain_comp4', 'comp1_1stwarning', 'comp2_1stwarning',
       'comp3_1stwarning', 'comp4_1stwarning', 'comp1_2ndwarning',
       'comp2_2ndwarning', 'comp3_2ndwarning', 'comp4_2ndwarning'],
      dtype='object')

In [38]:
final_data.to_csv('predictors_sql_data.csv', sep = ',',encoding='utf-8', index=False)

In [39]:
final_data.columns

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'model', 'age', 'error4', 'error3', 'error2', 'error1', 'Mantain_comp4',
       'Mantain_comp3', 'Mantain_comp2', 'Mantain_comp1', 'fail_comp4',
       'fail_comp3', 'fail_comp2', 'fail_comp1', 'DateTillMaintain_comp1',
       'DateTillMaintain_comp2', 'DateTillMaintain_comp3',
       'DateTillMaintain_comp4', 'DateTillFailure_comp1',
       'DateTillFailure_comp2', 'DateTillFailure_comp3',
       'DateTillFailure_comp4', 'RULtillFail_comp1', 'RULtillFail_comp2',
       'RULtillFail_comp3', 'RULtillFail_comp4', 'RULtillMaintain_comp1',
       'RULtillMaintain_comp2', 'RULtillMaintain_comp3',
       'RULtillMaintain_comp4', 'comp1_1stwarning', 'comp2_1stwarning',
       'comp3_1stwarning', 'comp4_1stwarning', 'comp1_2ndwarning',
       'comp2_2ndwarning', 'comp3_2ndwarning', 'comp4_2ndwarning'],
      dtype='object')

In [31]:
w1 = 30
w0 = 15
final_data['comp1_1stwarning'] = numpy.where(final_data['RULtillFail_comp1'] <= w1, 1, 0 )
final_data['comp2_1stwarning'] = numpy.where(final_data['RULtillFail_comp2'] <= w1, 1, 0 )
final_data['comp3_1stwarning'] = numpy.where(final_data['RULtillFail_comp3'] <= w1, 1, 0 )
final_data['comp4_1stwarning'] = numpy.where(final_data['RULtillFail_comp4'] <= w1, 1, 0 )
final_data['comp1_2ndwarning'] = final_data['comp1_1stwarning']
final_data['comp2_2ndwarning'] = final_data['comp2_1stwarning']
final_data['comp3_2ndwarning'] = final_data['comp3_1stwarning']
final_data['comp4_2ndwarning'] = final_data['comp4_1stwarning']


final_data.loc[final_data['RULtillFail_comp1'] <= w0, 'comp1_2ndwarning'] = 2
final_data.loc[final_data['RULtillFail_comp2'] <= w0, 'comp2_2ndwarning'] = 2
final_data.loc[final_data['RULtillFail_comp3'] <= w0, 'comp3_2ndwarning'] = 2
final_data.loc[final_data['RULtillFail_comp4'] <= w0, 'comp4_2ndwarning'] = 2

In [41]:
final_data.min()

datetime                  2014-06-01 06:00:00
machineID                                   1
volt                                        0
rotate                                      0
pressure                                    0
vibration                                   0
model                                       0
age                                         0
error4                                      0
error3                                      0
error2                                      0
error1                                      0
Mantain_comp4                               0
Mantain_comp3                               0
Mantain_comp2                               0
Mantain_comp1                               0
fail_comp4                                  0
fail_comp3                                  0
fail_comp2                                  0
fail_comp1                                  0
DateTillMaintain_comp1    2014-06-01 06:00:00
RULtillFail_comp1                 

In [40]:
final_data.count()

datetime                  8764644
machineID                 8764644
volt                      8764644
rotate                    8764644
pressure                  8764644
vibration                 8764644
model                     8764644
age                       8764644
error4                    8764644
error3                    8764644
error2                    8764644
error1                    8764644
Mantain_comp4             8764644
Mantain_comp3             8764644
Mantain_comp2             8764644
Mantain_comp1             8764644
fail_comp4                8764644
fail_comp3                8764644
fail_comp2                8764644
fail_comp1                8764644
DateTillMaintain_comp1    8764644
DateTillMaintain_comp2    8764603
DateTillMaintain_comp3    8764603
DateTillMaintain_comp4    8764585
DateTillFailure_comp1     8740280
DateTillFailure_comp2     8740528
DateTillFailure_comp3     8740614
DateTillFailure_comp4     8740267
RULtillFail_comp1         8740280
RULtillFail_co

In [45]:
final_data.loc[(final_data['RULtillFail_comp1'])<0, ['datetime','DateTillMaintain_comp1',
       'DateTillMaintain_comp2', 'DateTillMaintain_comp3',
       'DateTillMaintain_comp4', 'DateTillFailure_comp1',
       'DateTillFailure_comp2', 'DateTillFailure_comp3',
       'DateTillFailure_comp4']]

,datetime,DateTillMaintain_comp1,DateTillMaintain_comp2,DateTillMaintain_comp3,DateTillMaintain_comp4,DateTillFailure_comp1,DateTillFailure_comp2,DateTillFailure_comp3,DateTillFailure_comp4


In [57]:
#final_data=final_data.loc[final_data['volt']>0]
final_data = final_data.dropna()

In [66]:
final_data.columns

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'model', 'age', 'error4', 'error3', 'error2', 'error1', 'Mantain_comp4',
       'Mantain_comp3', 'Mantain_comp2', 'Mantain_comp1', 'fail_comp4',
       'fail_comp3', 'fail_comp2', 'fail_comp1', 'DateTillMaintain_comp1',
       'DateTillMaintain_comp2', 'DateTillMaintain_comp3',
       'DateTillMaintain_comp4', 'DateTillFailure_comp1',
       'DateTillFailure_comp2', 'DateTillFailure_comp3',
       'DateTillFailure_comp4', 'RULtillFail_comp1', 'RULtillFail_comp2',
       'RULtillFail_comp3', 'RULtillFail_comp4', 'RULtillMaintain_comp1',
       'RULtillMaintain_comp2', 'RULtillMaintain_comp3',
       'RULtillMaintain_comp4', 'comp1_1stwarning', 'comp2_1stwarning',
       'comp3_1stwarning', 'comp4_1stwarning', 'comp1_2ndwarning',
       'comp2_2ndwarning', 'comp3_2ndwarning', 'comp4_2ndwarning'],
      dtype='object')

In [68]:
train_data = final_data.loc[(final_data['datetime'])<'2015-08-01']
test_data = final_data.loc[(final_data['datetime'])>='2015-08-01']

In [67]:
train_data

,datetime,machineID,volt,rotate,pressure,vibration,model,age,error4,error3,...,RULtillMaintain_comp3,RULtillMaintain_comp4,comp1_1stwarning,comp2_1stwarning,comp3_1stwarning,comp4_1stwarning,comp1_2ndwarning,comp2_2ndwarning,comp3_2ndwarning,comp4_2ndwarning
4987350,2015-04-08 19:00:00,570,172.063971,355.749478,77.690885,39.208310,model3,19.0,0.0,0.0,...,11.0,11.0,1,1,0,1,2,2,0,2
8040992,2015-10-26 09:00:00,918,167.569744,433.647977,98.015338,27.045020,model3,0.0,0.0,0.0,...,21.0,21.0,1,1,1,1,1,1,1,1
2295295,2015-12-28 16:00:00,262,184.480340,501.672262,96.854326,40.380504,model4,20.0,0.0,0.0,...,14.0,14.0,1,1,1,1,2,2,2,2
5407158,2015-03-09 19:00:00,618,163.946017,363.310935,94.670197,49.911571,model4,7.0,0.0,0.0,...,11.0,11.0,1,1,0,1,2,2,0,2
2190443,2015-01-09 07:00:00,251,163.701948,440.768875,108.885708,25.426904,model1,9.0,0.0,0.0,...,23.0,23.0,1,1,1,1,1,1,1,1
3952254,2015-02-13 17:00:00,452,176.633816,444.158320,115.201762,44.640327,model4,4.0,0.0,0.0,...,13.0,13.0,1,1,1,1,2,2,2,2
2988571,2015-02-14 20:00:00,342,172.718198,378.178284,105.346819,43.563661,model2,15.0,0.0,0.0,...,10.0,10.0,1,1,1,1,2,2,2,2
3456273,2015-07-05 05:00:00,395,187.567560,446.140014,101.136013,35.818152,model3,17.0,0.0,0.0,...,1.0,1.0,1,1,1,1,2,2,2,2
1444726,2015-11-27 08:00:00,165,167.654651,542.104450,116.512410,46.435633,model4,19.0,0.0,0.0,...,22.0,22.0,1,1,1,1,1,1,1,1
2675037,2015-05-03 10:00:00,306,170.525226,495.563434,98.416565,38.447296,model1,11.0,0.0,0.0,...,20.0,20.0,1,1,1,1,1,1,1,1


In [58]:
final_data.loc[final_data['datetime']>'2015-05-01', ['datetime','DateTillMaintain_comp1',
       'DateTillMaintain_comp2', 'DateTillMaintain_comp3',
       'DateTillMaintain_comp4', 'DateTillFailure_comp1',
       'DateTillFailure_comp2', 'DateTillFailure_comp3',
       'DateTillFailure_comp4']]

,datetime,DateTillMaintain_comp1,DateTillMaintain_comp2,DateTillMaintain_comp3,DateTillMaintain_comp4,DateTillFailure_comp1,DateTillFailure_comp2,DateTillFailure_comp3,DateTillFailure_comp4
8255736,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
8229453,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
4295764,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
1527288,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
7449724,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
2193124,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
4663726,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
607383,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
8457239,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
1185609,2015-05-01 00:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00,2015-05-01 06:00:00
